In [ ]:
import pandas as pd
import sqlalchemy

In [2]:
data1 = pd.read_csv("../donnee_concatene/donnee.csv")

In [4]:
data1 = data1[data1["temperature_2m"].notna()]
data1.sample(2)

,Unnamed: 0.1,Unnamed: 0,date,temperature_2m,relative_humidity_2m,dew_point_2m,rain,precipitation_probability,precipitation,wind_speed_10m,...,visibility,weather_code,apparent_temperature,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,showers,island,ville,grille_meteo
223552,227008,10648,2022-05-16 16:00:00+00:00,26.850000,72.0,21.377195,0.0,NaN,0.2,17.298738,...,24140.0,51.0,28.904465,0.00,0.069438,0.988872,0.2,Mwali,"زيروداني Ziroudani, Djando, موهيلي Mohéli, Com...",-12.25_43.75
72990,74142,30870,2024-09-05 06:00:00+00:00,27.383501,64.0,19.973309,0.0,NaN,0.0,10.009036,...,24140.0,2.0,29.769150,0.15,0.337246,1.311926,0.0,Ngazidja,"Cembenoi-Sada-Djoumamlima, Mitsamiouli-Mboudé,...",-11.5_43.375


In [5]:
import os
import pandas as pd

class Pretraitement:

    def __init__(self, lien_dossier):
        self.dossier = lien_dossier
    
    # suppression des lignes vides et concatenation des dataframes 
    def supp_lignes_vides(self):
        dataframes = []

        for fichier in os.listdir(self.dossier):
            if fichier.endswith(".csv"):
                chemin = os.path.join(self.dossier, fichier)
                df = pd.read_csv(chemin)
                dataframes.append(df)

        data = pd.concat(dataframes, ignore_index=True)
        data = data[data["temperature_2m"].notna()]
        data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

        return data
    
    # creation du fichier csv qui contient tout les données 
    def creation_csv(self, lien):
        self.lien = lien 
        return self.supp_lignes_vides().to_csv(self.lien)
    
    # Ingection dans postgresQL 
    def injection_postgresql(self, host, port, database, user, password, table_name):
        from sqlalchemy import create_engine

        engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
        data = self.supp_lignes_vides()
        data.to_sql(table_name, engine, if_exists='replace', index=False)
    
    



In [6]:
donnee = Pretraitement("../data")
# donnee.creation_csv("ben.csv")
donne1 = donnee.supp_lignes_vides()
# b

In [7]:
donne1.info()

<class 'pandas.DataFrame'>
Index: 555048 entries, 576 to 562535
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   date                        555048 non-null  str    
 1   temperature_2m              555048 non-null  float64
 2   relative_humidity_2m        555048 non-null  float64
 3   dew_point_2m                555048 non-null  float64
 4   rain                        555035 non-null  float64
 5   precipitation_probability   70473 non-null   float64
 6   precipitation               555035 non-null  float64
 7   wind_speed_10m              555048 non-null  float64
 8   wind_direction_10m          555048 non-null  float64
 9   wind_speed_80m              555048 non-null  float64
 10  wind_speed_120m             555048 non-null  float64
 11  wind_speed_180m             366808 non-null  float64
 12  wind_direction_80m          555048 non-null  float64
 13  wind_direction_120m         

In [49]:
pd.read_csv("/home/ben/Bureau/climat_Comores/src/ben.csv")

,Unnamed: 0,date,temperature_2m,relative_humidity_2m,dew_point_2m,rain,precipitation_probability,precipitation,wind_speed_10m,wind_direction_10m,...,visibility,weather_code,apparent_temperature,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,showers,island,ville,grille_meteo
0,576,2021-03-23 00:00:00+00:00,10.351999,98.0,10.049539,NaN,NaN,NaN,2.160000,360.00000,...,22660.0,NaN,10.067563,0.00,NaN,0.025185,NaN,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
1,577,2021-03-23 01:00:00+00:00,10.351999,98.0,10.049539,0.0,NaN,0.0,1.609969,333.43503,...,22640.0,3.0,10.147776,0.00,0.000000,0.025185,0.0,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
2,578,2021-03-23 02:00:00+00:00,10.302000,99.0,10.151502,0.0,NaN,0.0,1.527351,315.00010,...,23100.0,3.0,10.139101,0.00,0.000000,0.012550,0.0,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
3,579,2021-03-23 03:00:00+00:00,10.251999,98.0,9.949777,0.0,NaN,0.0,1.018234,315.00010,...,24140.0,3.0,10.105595,0.00,0.000000,0.025017,0.0,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
4,580,2021-03-23 04:00:00+00:00,10.902000,97.0,10.444284,0.0,NaN,0.0,0.509117,224.99990,...,24140.0,2.0,10.972639,0.00,0.015300,0.039184,0.0,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555043,562531,2026-02-03 19:00:00+00:00,24.352000,81.0,20.880453,0.0,5.0,0.0,10.948973,297.40750,...,24140.0,2.0,27.070435,0.10,0.020412,0.578744,0.0,Mwali,"Fomboni, موهيلي Mohéli, Comores Komori جزر القمر",-12.25_43.75
555044,562532,2026-02-03 20:00:00+00:00,24.302000,82.0,21.031347,0.0,8.0,0.0,10.002560,300.25635,...,24140.0,2.0,27.238316,0.10,0.015849,0.546638,0.0,Mwali,"Fomboni, موهيلي Mohéli, Comores Komori جزر القمر",-12.25_43.75
555045,562533,2026-02-03 21:00:00+00:00,24.302000,82.0,21.031347,0.0,13.0,0.0,10.144082,297.47433,...,24140.0,2.0,27.217676,0.09,0.016254,0.546638,0.0,Mwali,"Fomboni, موهيلي Mohéli, Comores Komori جزر القمر",-12.25_43.75
555046,562534,2026-02-03 22:00:00+00:00,24.251999,82.0,20.982561,0.0,15.0,0.0,9.511088,299.47580,...,24140.0,3.0,27.234170,0.09,0.014375,0.545007,0.0,Mwali,"Fomboni, موهيلي Mohéli, Comores Komori جزر القمر",-12.25_43.75


#### lecture et manipulation 

In [53]:
donnee_clair = pd.read_csv("/home/ben/Bureau/climat_Comores/src/donnee.csv", index_col=0)
donnee_clair = donnee_clair.loc[:, ~donnee_clair.columns.str.contains("^Unnamed")]
donnee_clair.shape

(555048, 35)

In [54]:
donnee_clair = donnee_clair.drop_duplicates()
donnee_clair.drop_duplicates().shape

(555048, 35)

In [60]:
donnee_clair.dtypes

date                              str
temperature_2m                float64
relative_humidity_2m          float64
dew_point_2m                  float64
rain                          float64
precipitation_probability     float64
precipitation                 float64
wind_speed_10m                float64
wind_direction_10m            float64
wind_speed_80m                float64
wind_speed_120m               float64
wind_speed_180m               float64
wind_direction_80m            float64
wind_direction_120m           float64
wind_direction_180m           float64
wind_gusts_10m                float64
temperature_80m               float64
temperature_120m              float64
temperature_180m              float64
cloud_cover                   float64
cloud_cover_low               float64
cloud_cover_mid               float64
cloud_cover_high              float64
pressure_msl                  float64
surface_pressure              float64
visibility                    float64
weather_code

In [55]:
donnee_clair.describe()

,temperature_2m,relative_humidity_2m,dew_point_2m,rain,precipitation_probability,precipitation,wind_speed_10m,wind_direction_10m,wind_speed_80m,wind_speed_120m,...,cloud_cover_high,pressure_msl,surface_pressure,visibility,weather_code,apparent_temperature,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,showers
count,555048.000000,555048.000000,555048.000000,555035.000000,70473.000000,555035.000000,555048.000000,555048.000000,555048.000000,555048.000000,...,555035.000000,555048.000000,555048.000000,555048.000000,554998.000000,555048.000000,555048.000000,555035.000000,555048.000000,555035.000000
mean,24.521592,77.334872,20.170052,0.006888,14.804067,0.121698,12.574976,155.031093,15.020442,15.470531,...,26.943816,1014.162821,975.696691,22303.047772,13.428684,27.092403,0.096168,0.176583,0.737070,0.112977
std,4.447613,9.181619,4.184091,0.192500,20.790174,0.434572,7.655186,87.861666,9.210786,9.562977,...,39.352716,3.054644,60.091555,4895.064819,26.417530,6.277749,0.102101,0.198678,0.374133,0.370867
min,3.902000,32.000000,-2.183547,0.000000,0.000000,0.000000,0.000000,0.584628,0.000000,0.000000,...,-1.000000,1003.600000,770.688840,100.000000,0.000000,1.360477,0.000000,0.000000,0.000000,0.000000
25%,23.483500,71.000000,18.683947,0.000000,0.000000,0.000000,6.840000,92.202550,7.771331,7.928177,...,0.000000,1011.800000,967.286400,24140.000000,1.000000,24.811324,0.000000,0.015440,0.505885,0.000000
50%,25.270000,77.000000,20.896652,0.000000,5.000000,0.000000,10.883676,152.928010,13.009903,13.375249,...,5.000000,1014.100000,993.064800,24140.000000,2.000000,28.012420,0.070000,0.073229,0.719022,0.000000
75%,27.012000,83.000000,22.753060,0.000000,20.000000,0.000000,16.873980,189.462250,21.120682,21.849932,...,50.000000,1016.600000,1011.190300,24140.000000,3.000000,30.947193,0.160000,0.332100,0.935708,0.000000
max,35.759000,100.000000,28.286875,43.800000,100.000000,45.000000,53.096035,360.000000,62.938255,65.337296,...,101.000000,1024.000000,1023.400000,24140.000000,96.000000,42.504780,0.580000,0.862587,3.984451,21.300000


In [ ]:
validation = {
    "date": {
        "type": "str",
        "nullable": False,
        "unit": "ISO 8601 date (YYYY-MM-DD or datetime)"
    },

    "temperature_2m": {
        "type": float,
        "min_obs": 3.9,
        "max_obs": 35.759,
        "hard_min": -1,
        "hard_max": 50,
        "nullable": False,
        "unit": "°C"
    },

    "relative_humidity_2m": {
        "type": float,
        "min_obs": 32,
        "max_obs": 100,
        "hard_min": 0,
        "hard_max": 200,
        "nullable": False,
        "unit": "%"
    },

    "dew_point_2m": {
        "type": float,
        "min_obs": -2.183547,
        "max_obs": 28.286875,
        "hard_min": -5,
        "hard_max": 50,
        "nullable": False,
        "unit": "°C"
    },

    "rain": {
        "type": float,
        "min_obs": 0.0,
        "max_obs": 43.8,
        "hard_min": 0,
        "hard_max": 300,
        "nullable": True,
        "unit": "mm"
    },

    "precipitation_probability": {
        "type": float,
        "min_obs": 0,
        "max_obs": 100,
        "nullable": True,
        "unit": "%"
    },

    "precipitation": {
        "type": float,
        "min_obs": 0,
        "max_obs": 45.0,
        "hard_min": 0,
        "hard_max": 300,
        "nullable": True,
        "unit": "mm"
    },

    "wind_speed_10m": {
        "type": float,
        "min_obs": 0,
        "max_obs": 53.096035,
        "hard_min": 0,
        "hard_max": 100,
        "nullable": False,
        "unit": "km/h"
    },

    "wind_direction_10m": {
        "type": float,
        "min_obs": 0,
        "max_obs": 54,
        "hard_min": 0,
        "hard_max": 360,
        "nullable": False,
        "unit": "degrees (0–360)"
    },

    "wind_speed_80m": {
        "type": float,
        "min_obs": 0.584628,
        "max_obs": 62.938255,
        "nullable": True,
        "unit": "km/h"
    },

    "wind_speed_120m": {
        "type": float,
        "min_obs": 0,
        "max_obs": 65.337296,
        "nullable": True,
        "unit": "km/h"
    },

    "cloud_cover_high": {
        "type": float,
        "min_obs": -1.0,
        "max_obs": 101.0,
        "nullable": True,
        "unit": "%"
    },

    "pressure_msl": {
        "type": float,
        "min_obs": 1003.6,
        "max_obs": 1024,
        "hard_min": 950,
        "hard_max": 1050,
        "nullable": False,
        "unit": "hPa"
    },

    "surface_pressure": {
        "type": float,
        "min_obs": 770.688840,
        "max_obs": 1023.4,
        "hard_min": 950,
        "hard_max": 1050,
        "nullable": False,
        "unit": "hPa"
    },

    "visibility": {
        "type": float,
        "min_obs": 100.0,
        "max_obs": 24140.0,
        "hard_min": 0,
        "hard_max": 50000,
        "nullable": True,
        "unit": "meters"
    },

    "weather_code": {
        "type": int,
        "min_obs": 0,
        "max_obs": 96,
        "hard_min": 0,
        "hard_max": 99,
        "nullable": True,
        "unit": "WMO weather code"
    },

    "apparent_temperature": {
        "type": float,
        "min_obs": 1.360477,
        "max_obs": 42.504780,
        "hard_min": -5,
        "hard_max": 55,
        "nullable": False,
        "unit": "°C"
    },

    "evapotranspiration": {
        "type": float,
        "min_obs": 0,
        "max_obs": 0.58,
        "hard_min": 0,
        "hard_max": 10,
        "nullable": True,
        "unit": "mm/day"
    },

    "et0_fao_evapotranspiration": {
        "type": float,
        "min_obs": 0.0,
        "max_obs": 0.862587,
        "hard_min": 0,
        "hard_max": 10,
        "nullable": True,
        "unit": "mm/day"
    },

    "vapour_pressure_deficit": {
        "type": float,
        "min_obs": 0.0,
        "max_obs": 3.984451,
        "hard_min": 0,
        "hard_max": 50,
        "nullable": True,
        "unit": "hPa"
    },

    "showers": {
        "type": float,
        "min_obs": 0.0,
        "max_obs": 21.30,
        "hard_min": 0,
        "hard_max": 200,
        "nullable": True,
        "unit": "mm"
    },
    "island": {"type":str, "nullable":False},
    "ville": {"type":str, "nullable":False},
    "grille_meteo": {"type":str, "nullable":False},
}


In [ ]:
#donnee_clair.isna().sum()

In [45]:
donnee_clair[donnee_clair["precipitation_probability"].isna()]

,Unnamed: 0,date,temperature_2m,relative_humidity_2m,dew_point_2m,rain,precipitation_probability,precipitation,wind_speed_10m,wind_direction_10m,...,visibility,weather_code,apparent_temperature,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,showers,island,ville,grille_meteo
576,576,2021-03-23 00:00:00+00:00,10.351999,98.0,10.049539,NaN,NaN,NaN,2.160000,360.00000,...,22660.0,NaN,10.067563,0.00,NaN,0.025185,NaN,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
577,577,2021-03-23 01:00:00+00:00,10.351999,98.0,10.049539,0.0,NaN,0.0,1.609969,333.43503,...,22640.0,3.0,10.147776,0.00,0.000000,0.025185,0.0,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
578,578,2021-03-23 02:00:00+00:00,10.302000,99.0,10.151502,0.0,NaN,0.0,1.527351,315.00010,...,23100.0,3.0,10.139101,0.00,0.000000,0.012550,0.0,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
579,579,2021-03-23 03:00:00+00:00,10.251999,98.0,9.949777,0.0,NaN,0.0,1.018234,315.00010,...,24140.0,3.0,10.105595,0.00,0.000000,0.025017,0.0,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
580,580,2021-03-23 04:00:00+00:00,10.902000,97.0,10.444284,0.0,NaN,0.0,0.509117,224.99990,...,24140.0,2.0,10.972639,0.00,0.015300,0.039184,0.0,Ngazidja,"Cratère du Karthala, Ngazidja القمر الكبرى, Co...",-11.75_43.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557110,37846,2025-06-22 22:00:00+00:00,21.952000,76.0,17.528260,0.0,NaN,0.0,14.759999,180.00000,...,24140.0,2.0,22.502630,0.13,0.038777,0.632648,0.0,Mwali,"Fomboni, موهيلي Mohéli, Comores Komori جزر القمر",-12.25_43.75
557111,37847,2025-06-22 23:00:00+00:00,21.952000,75.0,17.318490,0.0,NaN,0.0,15.158522,184.08554,...,24140.0,2.0,22.353025,0.13,0.042253,0.659019,0.0,Mwali,"Fomboni, موهيلي Mohéli, Comores Komori جزر القمر",-12.25_43.75
557112,37848,2025-06-23 00:00:00+00:00,21.952000,74.0,17.106245,0.0,NaN,0.0,12.015589,171.38443,...,24140.0,2.0,22.719880,0.13,0.033894,0.685391,0.0,Mwali,"Fomboni, موهيلي Mohéli, Comores Komori جزر القمر",-12.25_43.75
557113,37849,2025-06-23 01:00:00+00:00,21.952000,75.0,17.318490,0.0,NaN,0.0,11.542478,176.42374,...,24140.0,2.0,22.880363,0.13,0.030359,0.659019,0.0,Mwali,"Fomboni, موهيلي Mohéli, Comores Komori جزر القمر",-12.25_43.75


In [ ]:
donnee_clair[donnee_clair["precipitation_probability"].isna()]

,Unnamed: 0.1,Unnamed: 0,date,temperature_2m,relative_humidity_2m,dew_point_2m,rain,precipitation_probability,precipitation,wind_speed_10m,...,visibility,weather_code,apparent_temperature,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,showers,island,ville,grille_meteo
471068,477980,1988,2021-05-20 20:00:00+00:00,23.675499,67.0,17.186598,0.0,NaN,0.0,22.264770,...,24140.0,0.0,22.981810,0.0,0.091071,0.965528,0.0,Mwali,"جويزي Djoiezi, Moili Mdjini, موهيلي Mohéli, Co...",-12.375_43.75
220884,224340,7980,2022-01-25 12:00:00+00:00,27.500000,80.0,23.740221,0.0,NaN,2.8,28.467329,...,12060.0,81.0,30.367645,0.0,0.475663,0.733565,2.8,Mwali,"زيروداني Ziroudani, Djando, موهيلي Mohéli, Com...",-12.25_43.75


In [ ]:
donne1.columns

Index(['Unnamed: 0', 'date', 'temperature_2m', 'relative_humidity_2m',
       'dew_point_2m', 'rain', 'precipitation_probability', 'precipitation',
       'wind_speed_10m', 'wind_direction_10m', 'wind_speed_80m',
       'wind_speed_120m', 'wind_speed_180m', 'wind_direction_80m',
       'wind_direction_120m', 'wind_direction_180m', 'wind_gusts_10m',
       'temperature_80m', 'temperature_120m', 'temperature_180m',
       'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
       'pressure_msl', 'surface_pressure', 'visibility', 'weather_code',
       'apparent_temperature', 'evapotranspiration',
       'et0_fao_evapotranspiration', 'vapour_pressure_deficit', 'showers',
       'island', 'ville', 'grille_meteo'],
      dtype='str')

In [ ]:
donne1.dtypes

Unnamed: 0                      int64
date                              str
temperature_2m                float64
relative_humidity_2m          float64
dew_point_2m                  float64
rain                          float64
precipitation_probability     float64
precipitation                 float64
wind_speed_10m                float64
wind_direction_10m            float64
wind_speed_80m                float64
wind_speed_120m               float64
wind_speed_180m               float64
wind_direction_80m            float64
wind_direction_120m           float64
wind_direction_180m           float64
wind_gusts_10m                float64
temperature_80m               float64
temperature_120m              float64
temperature_180m              float64
cloud_cover                   float64
cloud_cover_low               float64
cloud_cover_mid               float64
cloud_cover_high              float64
pressure_msl                  float64
surface_pressure              float64
visibility  